# new

In [3]:
# ========== Celda única para Jupyter: generar SVG con capas y Braille ==========
# Pega todo esto en una celda del notebook y ejecútalo (Shift+Enter).
# Requisitos: svgwrite, numpy (instalar con pip si no lo tienes).

import json
from pathlib import Path
import math
import numpy as np
import svgwrite
from IPython.display import SVG, display

# -----------------------
# 1) Preparar/leer params.json
# -----------------------
# Nombre del fichero de parámetros
PARAMS_FILE = "params.json"

# Si no existe params.json creamos uno de ejemplo para que lo edites
default_params = {
  "fig_size_mm": [173.0, 113.0],
  "mm_per_inch": 23.4555555,
  "xlim": [-7.0, 7.0],
  "ylim": [-7.0, 7.0],
  "tick_step": 0.5,
  "grid_stroke_mm": 0.25,
  "axis_stroke_mm": 0.6,
  "curve_stroke_mm": 0.9,
  "marker_edge_stroke_mm": 0.2,
  "marker_shapes": ["o", "s", "^"],
  "marker_sizes_mm": [3.0, 3.0, 3.5],
  # funciones como strings (usar 'np' si necesitas funciones numpy)
  "functions": ["x", "x**2", "x**3"],
  "curve_styles": ["solid", "dash", "dot"],
  "n_curve_samples": 800,
  # si "adaptive_default" usa la muestreadora incluida; también puedes poner una lista de listas
  "marker_xs": "adaptive_default",
  "braille_labels": [
    {
      "text": "Figura 1",
      "position_mm": [-70.0, -50.0],
      "dot_diameter_mm": 1.5,
      "dot_height_mm": 0.8,
      "dot_spacing_mm": 2.5,
      "char_spacing_mm": 3.0,
      "line_spacing_mm": 4.0
    }
  ],
  "output_svg": "A5_output_layers_from_params.svg"
}

# escribir ejemplo si no existe
p = Path(PARAMS_FILE)
if not p.exists():
    p.write_text(json.dumps(default_params, indent=2, ensure_ascii=False))
    print(f"No se encontró '{PARAMS_FILE}'. Se ha creado un ejemplo. Edita '{PARAMS_FILE}' y vuelve a ejecutar la celda.")
# leer params
params = json.loads(p.read_text(encoding="utf8"))

# -----------------------
# 2) Helpers: mapeos y muestreo
# -----------------------

# Mapa Braille básico (a-z)
BRAILLE_ALPHA = {
    'a': (1,), 'b': (1,2), 'c': (1,4), 'd': (1,4,5), 'e': (1,5),
    'f': (1,2,4), 'g': (1,2,4,5), 'h': (1,2,5), 'i': (2,4), 'j': (2,4,5),
    'k': (1,3), 'l': (1,2,3), 'm': (1,3,4), 'n': (1,3,4,5), 'o': (1,3,5),
    'p': (1,2,3,4), 'q': (1,2,3,4,5), 'r': (1,2,3,5), 's': (2,3,4), 't': (2,3,4,5),
    'u': (1,3,6), 'v': (1,2,3,6), 'w': (2,4,5,6), 'x': (1,3,4,6), 'y': (1,3,4,5,6), 'z': (1,3,5,6),
}
CAPITAL_SIGN = (6,)
NUMBER_SIGN = (3,4,5,6)
SPACE = ()
DIGIT_TO_ALPHA = { '1':'a','2':'b','3':'c','4':'d','5':'e','6':'f','7':'g','8':'h','9':'i','0':'j' }

def char_to_cells(ch):
    """Retorna lista de celdas Braille para carácter ch."""
    if ch == ' ': return [SPACE]
    if ch in [',', '.', ';', '-', '_', '/', '(', ')', ':']: return [SPACE]
    if ch.lower() in BRAILLE_ALPHA:
        if ch.isupper():
            return [CAPITAL_SIGN, BRAILLE_ALPHA[ch.lower()]]
        else:
            return [BRAILLE_ALPHA[ch.lower()]]
    return [SPACE]

def text_to_cells(text):
    """Convierte texto a secuencia de celdas Braille, con soporte para números."""
    cells = []
    i = 0
    while i < len(text):
        ch = text[i]
        if ch.isdigit():
            cells.append(NUMBER_SIGN)
            while i < len(text) and text[i].isdigit():
                cells.extend(char_to_cells(DIGIT_TO_ALPHA[text[i]]))
                i += 1
            continue
        cells.extend(char_to_cells(ch))
        i += 1
    return cells

def make_default_marker_xs(xlim):
    """Muestreo adaptativo por defecto (similar a ejemplos previos)."""
    xs1 = np.linspace(xlim[0], xlim[1], 35)
    xs2 = np.concatenate([
        np.linspace(xlim[0], -3.0, 1),
        np.linspace(-3.0, -2.0, 10),
        np.linspace(-2.0, -1.0, 7),
        np.linspace(-1.0, 1.0, 7),
        np.linspace(1.0, 2.0, 7),
        np.linspace(2.0, 3.0, 10),
        np.linspace(3.0, xlim[1], 1)
    ])
    xs3 = np.concatenate([
        np.linspace(xlim[0], -2.0, 1),
        np.linspace(-2.0, -1.5, 10),
        np.linspace(-1.5, -1.0, 6),
        np.linspace(-1.0, 1.0, 6),
        np.linspace(1.0, 1.5, 6),
        np.linspace(1.5, 2.0, 10),
        np.linspace(2.0, xlim[1], 1)
    ])
    return [xs1, xs2, xs3]

# -----------------------
# 3) Geometría → coordenadas SVG (mm)
# -----------------------
def data_to_svg_coords(x, y, xlim, ylim, width_mm, height_mm):
    """
    Mapear (x,y) datos -> coordenadas SVG en mm.
    Origen SVG: esquina superior izquierda. Sistema de datos centrado mapeado al rectángulo físico.
    """
    x0, x1 = xlim
    y0, y1 = ylim
    fx = (x - x0) / (x1 - x0)
    fy = (y - y0) / (y1 - y0)
    sx = fx * width_mm
    sy = (1 - fy) * height_mm
    return sx, sy

def svg_stroke_dash(style_name):
    if style_name == "solid": return None
    if style_name == "dash": return "6,3"
    if style_name == "dot": return "1,3"
    return None

# -----------------------
# 4) Render Braille -> grupo SVG (relativo)
# -----------------------
def render_braille_group(dwg, text, dot_diameter_mm=1.5, dot_spacing_mm=2.5,
                         char_spacing_mm=3.0, line_spacing_mm=4.0, fill_color="#000000"):
    """
    Crea y devuelve un grupo SVG (svgwrite container) con círculos que representan las celdas Braille.
    Este grupo está en coordenadas relativas (0,0) = origen proporcionado por el caller.
    """
    g = dwg.g()
    lines = text.split('\n')
    cursor_y = 0.0
    for line in lines:
        cells = text_to_cells(line)
        cursor_x = 0.0
        for cell in cells:
            if cell == SPACE:
                cursor_x += char_spacing_mm
                continue
            for d in cell:
                col = 0 if d in (1,2,3) else 1
                row_map = {1:0,2:1,3:2,4:0,5:1,6:2}
                row = row_map[d]
                x_offset = (col - 0.5) * dot_spacing_mm
                y_offset = (1 - row) * dot_spacing_mm
                cx = cursor_x + x_offset
                cy = cursor_y + y_offset
                g.add(dwg.circle(center=(f"{cx}mm", f"{cy}mm"), r=f"{(dot_diameter_mm/2.0):.3f}mm",
                                 fill=fill_color, stroke="none"))
            cursor_x += char_spacing_mm
        cursor_y += line_spacing_mm
    return g

# -----------------------
# 5) Construir SVG con capas
# -----------------------
def build_svg_from_params(params):
    # leer parámetros principales
    fig_w_mm, fig_h_mm = params.get("fig_size_mm", [173.0, 113.0])
    xlim = tuple(params.get("xlim", [-7.0, 7.0]))
    ylim = tuple(params.get("ylim", [-7.0, 7.0]))
    tick_step = params.get("tick_step", 0.5)
    output_svg = params.get("output_svg", "output.svg")

    # crear documento SVG con tamaño físico en mm
    dwg = svgwrite.Drawing(filename=output_svg, size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"), profile='tiny')

    # CAPA: plate (fondo)
    plate = dwg.g(id="plate", **{"inkscape:groupmode":"layer", "inkscape:label":"Plate"})
    plate.add(dwg.rect(insert=(0,0), size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"), fill="#ffffff"))
    dwg.add(plate)

    # CAPA: grid
    layer_grid = dwg.g(id="grid", **{"inkscape:groupmode":"layer", "inkscape:label":"Grid"})
    grid_stroke_mm = params.get("grid_stroke_mm", 0.25)
    xticks = np.arange(xlim[0], xlim[1] + 1e-9, tick_step)
    yticks = np.arange(ylim[0], ylim[1] + 1e-9, tick_step)
    for xv in xticks:
        sx1, sy1 = data_to_svg_coords(xv, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xv, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
        layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                stroke="#e6e6e6", stroke_width=f"{grid_stroke_mm}mm"))
    for yv in yticks:
        sx1, sy1 = data_to_svg_coords(xlim[0], yv, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xlim[1], yv, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                stroke="#f5f5f5", stroke_width=f"{grid_stroke_mm}mm"))
    dwg.add(layer_grid)

    # CAPA: axes
    layer_axes = dwg.g(id="axes", **{"inkscape:groupmode":"layer", "inkscape:label":"Axes"})
    axis_stroke_mm = params.get("axis_stroke_mm", 0.6)
    sx1, sy1 = data_to_svg_coords(xlim[0], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(xlim[1], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
    layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    sx1, sy1 = data_to_svg_coords(0.0, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(0.0, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
    layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    dwg.add(layer_axes)

    # CAPA: curves
    layer_curves = dwg.g(id="curves", **{"inkscape:groupmode":"layer", "inkscape:label":"Curves"})
    funcs_expr = params.get("functions", ["x"])
    curve_styles = params.get("curve_styles", ["solid"]*len(funcs_expr))
    n_samples = params.get("n_curve_samples", 800)

    # crear funciones evaluables a partir de strings (usa eval restringido)
    funcs = []
    for expr in funcs_expr:
        expr_str = str(expr)
        def make_func(expression):
            def f(x):
                # Nota de seguridad: se usa eval restringido. No ejecutar params.json de fuentes no confiables.
                return eval(expression, {"np": np, "x": x, "__builtins__": {}})
            return f
        funcs.append(make_func(expr_str))

    x_cont = np.linspace(xlim[0], xlim[1], n_samples)
    curve_stroke_mm = params.get("curve_stroke_mm", 0.9)
    for i, f in enumerate(funcs):
        ys = f(x_cont)
        pts_svg = [ data_to_svg_coords(xv, yv, xlim, ylim, fig_w_mm, fig_h_mm) for xv, yv in zip(x_cont, ys) ]
        pts_str = [ (f"{px:.6f}mm", f"{py:.6f}mm") for px,py in pts_svg ]
        dash = svg_stroke_dash(curve_styles[i] if i < len(curve_styles) else "solid")
        stroke_kwargs = {"stroke":"#222222", "fill":"none", "stroke_width":f"{curve_stroke_mm}mm"}
        if dash:
            stroke_kwargs["stroke_dasharray"] = dash
        layer_curves.add(dwg.polyline(points=[(x,y) for x,y in pts_str], **stroke_kwargs))
    dwg.add(layer_curves)

    # CAPA: markers
    layer_markers = dwg.g(id="markers", **{"inkscape:groupmode":"layer", "inkscape:label":"Markers"})
    marker_shapes = params.get("marker_shapes", ["o"])
    marker_sizes = params.get("marker_sizes_mm", [3.0])
    marker_xs_param = params.get("marker_xs", "adaptive_default")
    if marker_xs_param == "adaptive_default":
        marker_xs = make_default_marker_xs(xlim)
    else:
        marker_xs = marker_xs_param
    marker_edge_stroke_mm = params.get("marker_edge_stroke_mm", 0.2)
    for i, f in enumerate(funcs):
        xs = np.array(marker_xs[i]) if i < len(marker_xs) else np.array([])
        ys = f(xs) if xs.size else np.array([])
        shape = marker_shapes[i] if i < len(marker_shapes) else "o"
        size_mm = marker_sizes[i] if i < len(marker_sizes) else 3.0
        for xm, ym in zip(xs, ys):
            sx, sy = data_to_svg_coords(float(xm), float(ym), xlim, ylim, fig_w_mm, fig_h_mm)
            if shape == 'o':
                layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                             r=f"{(size_mm/2.0):.3f}mm",
                                             fill="#ffffff", stroke="#000000",
                                             stroke_width=f"{marker_edge_stroke_mm}mm"))
            elif shape == 's':
                half = size_mm/2.0
                x0 = sx - half
                y0 = sy - half
                layer_markers.add(dwg.rect(insert=(f"{x0}mm", f"{y0}mm"),
                                           size=(f"{size_mm}mm", f"{size_mm}mm"),
                                           fill="#ffffff", stroke="#000000",
                                           stroke_width=f"{marker_edge_stroke_mm}mm"))
            elif shape == '^':
                a = size_mm
                h = (math.sqrt(3)/2.0) * a
                v1 = (sx, sy - 2*h/3.0)
                v2 = (sx - a/2.0, sy + h/3.0)
                v3 = (sx + a/2.0, sy + h/3.0)
                layer_markers.add(dwg.polygon(points=[(f"{v1[0]}mm", f"{v1[1]}mm"),
                                                      (f"{v2[0]}mm", f"{v2[1]}mm"),
                                                      (f"{v3[0]}mm", f"{v3[1]}mm")],
                                              fill="#ffffff", stroke="#000000", stroke_width=f"{marker_edge_stroke_mm}mm"))
            else:
                layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                             r=f"{(size_mm/2.0):.3f}mm",
                                             fill="#ffffff", stroke="#000000",
                                             stroke_width=f"{marker_edge_stroke_mm}mm"))
    dwg.add(layer_markers)

    # CAPA: ticks
    layer_ticks = dwg.g(id="ticks", **{"inkscape:groupmode":"layer", "inkscape:label":"Ticks"})
    for yv in yticks:
        sx1, sy1 = data_to_svg_coords(0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(-0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                 stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    for xv in xticks:
        sx1, sy1 = data_to_svg_coords(xv, 0.12, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xv, -0.12, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                 stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    dwg.add(layer_ticks)

    # CAPA: braille (cada etiqueta en subgrupo con transform)
    layer_braille = dwg.g(id="braille", **{"inkscape:groupmode":"layer", "inkscape:label":"Braille"})
    for lbl in params.get("braille_labels", []):
        text = lbl.get("text", "")
        pos = lbl.get("position_mm", [0.0, 0.0])
        d_diam = float(lbl.get("dot_diameter_mm", 1.5))
        d_sp = float(lbl.get("dot_spacing_mm", 2.5))
        c_sp = float(lbl.get("char_spacing_mm", 3.0))
        l_sp = float(lbl.get("line_spacing_mm", 4.0))
        sub_id = f"braille_{text.replace(' ', '_')}"
        sub = dwg.g(id=sub_id, **{"inkscape:groupmode":"layer", "inkscape:label":f"Braille: {text}"})
        braille_group = render_braille_group(dwg, text,
                                            dot_diameter_mm=d_diam,
                                            dot_spacing_mm=d_sp,
                                            char_spacing_mm=c_sp,
                                            line_spacing_mm=l_sp)
        # calcular transformación: coordenadas centradas -> coordenadas svg
        ox_mm = pos[0] + fig_w_mm / 2.0
        oy_mm = fig_h_mm / 2.0 - pos[1]
        trans = f"translate({ox_mm},{oy_mm})"
        # envolver y añadir
        sub.add(dwg.g(braille_group.elements, transform=trans))
        layer_braille.add(sub)
    dwg.add(layer_braille)

    # guardar svg
    dwg.save()
    print(f"SVG guardado en: {output_svg}")

    # intentar mostrar inline (si Jupyter puede renderizar)
    try:
        display(SVG(output_svg))
    except Exception:
        print("No se pudo mostrar inline; abre el archivo manualmente:", output_svg)

# -----------------------
# 6) Ejecutar
# -----------------------
output_file = params.get("output_svg", "A5_output_layers_from_params.svg")
# crear los arrays de ticks una vez (los usa la función)
xticks = np.arange(params.get("xlim", [-7,7])[0], params.get("xlim", [-7,7])[1] + 1e-9, params.get("tick_step", 0.5))
yticks = np.arange(params.get("ylim", [-7,7])[0], params.get("ylim", [-7,7])[1] + 1e-9, params.get("tick_step", 0.5))

build_svg_from_params(params)
# ====================================================================================


ValueError: Invalid attribute 'inkscape:groupmode' for svg-element <g>.

# thi is

In [2]:
#!/usr/bin/env python3
"""
generate_svg_from_params.py

Lee un archivo JSON de parámetros (params.json) y genera un SVG con capas:
 - Plate (fondo)
 - Grid
 - Axes
 - Curves
 - Markers
 - Ticks
 - Braille (cada etiqueta en subgrupo)

Requisitos:
    pip install svgwrite numpy

Uso:
    python generate_svg_from_params.py params.json
"""

import sys
import json
import math
import numpy as np
import svgwrite
from pathlib import Path

# -----------------------
# UTILIDADES / BRAILLE
# -----------------------

# Mapa Braille básico (letters a-z)
BRAILLE_ALPHA = {
    'a': (1,), 'b': (1,2), 'c': (1,4), 'd': (1,4,5), 'e': (1,5),
    'f': (1,2,4), 'g': (1,2,4,5), 'h': (1,2,5), 'i': (2,4), 'j': (2,4,5),
    'k': (1,3), 'l': (1,2,3), 'm': (1,3,4), 'n': (1,3,4,5), 'o': (1,3,5),
    'p': (1,2,3,4), 'q': (1,2,3,4,5), 'r': (1,2,3,5), 's': (2,3,4), 't': (2,3,4,5),
    'u': (1,3,6), 'v': (1,2,3,6), 'w': (2,4,5,6), 'x': (1,3,4,6), 'y': (1,3,4,5,6), 'z': (1,3,5,6),
}
CAPITAL_SIGN = (6,)
NUMBER_SIGN = (3,4,5,6)
SPACE = ()
DIGIT_TO_ALPHA = { '1':'a','2':'b','3':'c','4':'d','5':'e','6':'f','7':'g','8':'h','9':'i','0':'j' }

def char_to_cells(ch):
    """Mapa básico: retorna lista de celdas (tuplas de puntos) para ch."""
    if ch == ' ':
        return [SPACE]
    if ch in [',', '.', ';', '-', '_', '/', '(', ')', ':']:
        return [SPACE]
    if ch.lower() in BRAILLE_ALPHA:
        if ch.isupper():
            return [CAPITAL_SIGN, BRAILLE_ALPHA[ch.lower()]]
        else:
            return [BRAILLE_ALPHA[ch.lower()]]
    return [SPACE]

def text_to_cells(text):
    """Convierte texto a secuencia de celdas Braille, maneja números."""
    cells = []
    i = 0
    while i < len(text):
        ch = text[i]
        if ch.isdigit():
            cells.append(NUMBER_SIGN)
            while i < len(text) and text[i].isdigit():
                cells.extend(char_to_cells(DIGIT_TO_ALPHA[text[i]]))
                i += 1
            continue
        cells.extend(char_to_cells(ch))
        i += 1
    return cells

# -----------------------
# GEOM → SVG helpers
# -----------------------

def data_to_svg_coords(x, y, xlim, ylim, width_mm, height_mm):
    """
    Mapea (x,y) en datos a coordenadas SVG en mm.
    Sistema de datos mapeado linealmente al rectángulo físico [0,width_mm]x[0,height_mm].
    """
    x0, x1 = xlim
    y0, y1 = ylim
    fx = (x - x0) / (x1 - x0)
    fy = (y - y0) / (y1 - y0)
    sx = fx * width_mm
    sy = (1 - fy) * height_mm
    return sx, sy

def svg_stroke_dash(style_name):
    if style_name == "solid": return None
    if style_name == "dash": return "6,3"
    if style_name == "dot": return "1,3"
    return None

# -----------------------
# RENDER BRAILLE TO SVG
# -----------------------

def render_braille_to_group(dwg, text, origin_mm, dot_diameter_mm=1.5, dot_spacing_mm=2.5,
                            char_spacing_mm=3.0, line_spacing_mm=4.0, fill_color="#000000"):
    """
    Devuelve un grupo (svgwrite container) con los círculos que representan el Braille.
    origin_mm está en coordenadas centradas (-w/2..w/2, -h/2..h/2) y la función convertirá
    a coordenadas SVG en mm cuando lo insertemos.
    """
    g = dwg.g()
    ox, oy = origin_mm
    # NOTE: caller must translate el grupo a coordenadas svg adecuadas (alternativa: calcular en caller)
    # Aquí dibujamos en coordenadas relativas: (0,0) corresponde al origin_mm en el sistema centrado.
    lines = text.split('\n')
    cursor_y = 0.0
    for line in lines:
        cells = text_to_cells(line)
        cursor_x = 0.0
        for cell in cells:
            if cell == SPACE:
                cursor_x += char_spacing_mm
                continue
            for d in cell:
                # map dot index -> offset
                col = 0 if d in (1,2,3) else 1
                row_map = {1:0,2:1,3:2,4:0,5:1,6:2}
                row = row_map[d]
                x_offset = (col - 0.5) * dot_spacing_mm
                y_offset = (1 - row) * dot_spacing_mm  # row0 top, row2 bottom
                cx = cursor_x + x_offset
                cy = cursor_y + y_offset
                # circle center at (cx, cy) in mm relative to origin
                g.add(dwg.circle(center=(f"{cx}mm", f"{cy}mm"),
                                 r=f"{dot_diameter_mm/2.0:.3f}mm",
                                 fill=fill_color, stroke="none"))
            cursor_x += char_spacing_mm
        cursor_y += line_spacing_mm
    # The group is drawn centered at (0,0) — caller should transform/translate to absolute svg coords.
    return g

# -----------------------
# MAIN: lee params y genera svg
# -----------------------

def make_default_marker_xs(xlim):
    """Muestreo adaptativo por defecto (similar al que usabas). Devuelve lista por función."""
    xs1 = np.linspace(xlim[0], xlim[1], 35)
    xs2 = np.concatenate([
        np.linspace(xlim[0], -3.0, 1),
        np.linspace(-3.0, -2.0, 10),
        np.linspace(-2.0, -1.0, 7),
        np.linspace(-1.0, 1.0, 7),
        np.linspace(1.0, 2.0, 7),
        np.linspace(2.0, 3.0, 10),
        np.linspace(3.0, xlim[1], 1)
    ])
    xs3 = np.concatenate([
        np.linspace(xlim[0], -2.0, 1),
        np.linspace(-2.0, -1.5, 10),
        np.linspace(-1.5, -1.0, 6),
        np.linspace(-1.0, 1.0, 6),
        np.linspace(1.0, 1.5, 6),
        np.linspace(1.5, 2.0, 10),
        np.linspace(2.0, xlim[1], 1)
    ])
    return [xs1, xs2, xs3]

def load_params(path):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"params file not found: {path}")
    with p.open("r", encoding="utf8") as fh:
        return json.load(fh)

def build_svg_from_params(params):
    # read common params
    fig_w_mm, fig_h_mm = params.get("fig_size_mm", [173.0, 113.0])
    xlim = tuple(params.get("xlim", [-7.0, 7.0]))
    ylim = tuple(params.get("ylim", [-7.0, 7.0]))
    tick_step = params.get("tick_step", 0.5)
    output_svg = params.get("output_svg", "output.svg")

    # create svgwrite drawing with physical mm size
    dwg = svgwrite.Drawing(filename=output_svg, size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"), profile='tiny')

    # layers (groups) with inkscape-compatible attributes
    # we will add groups and then fill them
    inkscape_extra = {"inkscape:groupmode": "layer"}  # note: svgwrite won't add namespace by default but inkscape reads the attributes

    # 1) plate (background)
    plate = dwg.g(id="plate", **{"inkscape:groupmode":"layer", "inkscape:label":"Plate"})
    plate.add(dwg.rect(insert=(0,0), size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"), fill="#ffffff"))
    dwg.add(plate)

    # 2) grid
    layer_grid = dwg.g(id="grid", **{"inkscape:groupmode":"layer", "inkscape:label":"Grid"})
    grid_stroke_mm = params.get("grid_stroke_mm", 0.25)
    xticks = np.arange(xlim[0], xlim[1] + 1e-9, tick_step)
    yticks = np.arange(ylim[0], ylim[1] + 1e-9, tick_step)
    for xv in xticks:
        sx1, sy1 = data_to_svg_coords(xv, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xv, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
        layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                stroke="#e6e6e6", stroke_width=f"{grid_stroke_mm}mm"))
    for yv in yticks:
        sx1, sy1 = data_to_svg_coords(xlim[0], yv, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xlim[1], yv, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                stroke="#f5f5f5", stroke_width=f"{grid_stroke_mm}mm"))
    dwg.add(layer_grid)

    # 3) axes
    layer_axes = dwg.g(id="axes", **{"inkscape:groupmode":"layer", "inkscape:label":"Axes"})
    axis_stroke_mm = params.get("axis_stroke_mm", 0.6)
    sx1, sy1 = data_to_svg_coords(xlim[0], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(xlim[1], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
    layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    sx1, sy1 = data_to_svg_coords(0.0, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(0.0, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
    layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    dwg.add(layer_axes)

    # 4) curves
    layer_curves = dwg.g(id="curves", **{"inkscape:groupmode":"layer", "inkscape:label":"Curves"})
    funcs_expr = params.get("functions", ["x"])
    curve_styles = params.get("curve_styles", ["solid"]*len(funcs_expr))
    n_samples = params.get("n_curve_samples", 800)

    # build callable functions from strings (evaluated safely with numpy)
    funcs = []
    for expr in funcs_expr:
        expr_str = str(expr)
        def make_func(expression):
            def f(x):
                return eval(expression, {"np": np, "x": x, "__builtins__": {}})
            return f
        funcs.append(make_func(expr_str))

    x_cont = np.linspace(xlim[0], xlim[1], n_samples)
    curve_stroke_mm = params.get("curve_stroke_mm", 0.9)
    for i, f in enumerate(funcs):
        ys = f(x_cont)
        pts_svg = [ data_to_svg_coords(xv, yv, xlim, ylim, fig_w_mm, fig_h_mm) for xv, yv in zip(x_cont, ys) ]
        pts_str = [ (f"{px:.6f}mm", f"{py:.6f}mm") for px,py in pts_svg ]
        dash = svg_stroke_dash(curve_styles[i] if i < len(curve_styles) else "solid")
        stroke_kwargs = {"stroke":"#222222", "fill":"none", "stroke_width":f"{curve_stroke_mm}mm"}
        if dash:
            stroke_kwargs["stroke_dasharray"] = dash
        layer_curves.add(dwg.polyline(points=[(x,y) for x,y in pts_str], **stroke_kwargs))
    dwg.add(layer_curves)

    # 5) markers
    layer_markers = dwg.g(id="markers", **{"inkscape:groupmode":"layer", "inkscape:label":"Markers"})
    marker_shapes = params.get("marker_shapes", ["o"])
    marker_sizes = params.get("marker_sizes_mm", [3.0])
    # marker_xs: either "adaptive_default" or explicit list
    marker_xs_param = params.get("marker_xs", "adaptive_default")
    if marker_xs_param == "adaptive_default":
        marker_xs = make_default_marker_xs(xlim)
    else:
        # expect a list of lists in params
        marker_xs = marker_xs_param

    marker_edge_stroke_mm = params.get("marker_edge_stroke_mm", 0.2)
    for i, f in enumerate(funcs):
        xs = np.array(marker_xs[i]) if i < len(marker_xs) else np.array([])
        ys = f(xs) if xs.size else np.array([])
        shape = marker_shapes[i] if i < len(marker_shapes) else "o"
        size_mm = marker_sizes[i] if i < len(marker_sizes) else 3.0
        for xm, ym in zip(xs, ys):
            sx, sy = data_to_svg_coords(float(xm), float(ym), xlim, ylim, fig_w_mm, fig_h_mm)
            if shape == 'o':
                layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                             r=f"{(size_mm/2.0):.3f}mm",
                                             fill="#ffffff", stroke="#000000",
                                             stroke_width=f"{marker_edge_stroke_mm}mm"))
            elif shape == 's':
                half = size_mm/2.0
                x0 = sx - half
                y0 = sy - half
                layer_markers.add(dwg.rect(insert=(f"{x0}mm", f"{y0}mm"),
                                           size=(f"{size_mm}mm", f"{size_mm}mm"),
                                           fill="#ffffff", stroke="#000000",
                                           stroke_width=f"{marker_edge_stroke_mm}mm"))
            elif shape == '^':
                a = size_mm
                h = (math.sqrt(3)/2.0) * a
                v1 = (sx, sy - 2*h/3.0)
                v2 = (sx - a/2.0, sy + h/3.0)
                v3 = (sx + a/2.0, sy + h/3.0)
                layer_markers.add(dwg.polygon(points=[(f"{v1[0]}mm", f"{v1[1]}mm"),
                                                      (f"{v2[0]}mm", f"{v2[1]}mm"),
                                                      (f"{v3[0]}mm", f"{v3[1]}mm")],
                                              fill="#ffffff", stroke="#000000", stroke_width=f"{marker_edge_stroke_mm}mm"))
            else:
                layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                             r=f"{(size_mm/2.0):.3f}mm",
                                             fill="#ffffff", stroke="#000000",
                                             stroke_width=f"{marker_edge_stroke_mm}mm"))
    dwg.add(layer_markers)

    # 6) ticks (small axis marks)
    layer_ticks = dwg.g(id="ticks", **{"inkscape:groupmode":"layer", "inkscape:label":"Ticks"})
    tick_len_mm = 0.8
    for yv in yticks:
        sx1, sy1 = data_to_svg_coords(0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(-0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                 stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    for xv in xticks:
        sx1, sy1 = data_to_svg_coords(xv, 0.12, xlim, ylim, fig_w_mm, fig_h_mm)
        sx2, sy2 = data_to_svg_coords(xv, -0.12, xlim, ylim, fig_w_mm, fig_h_mm)
        layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                                 stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
    dwg.add(layer_ticks)

    # 7) braille labels: each label as sub-group (translated to absolute svg coords)
    layer_braille = dwg.g(id="braille", **{"inkscape:groupmode":"layer", "inkscape:label":"Braille"})
    for lbl in params.get("braille_labels", []):
        text = lbl.get("text", "")
        pos = lbl.get("position_mm", [0.0, 0.0])  # coordenadas centradas (-w/2..w/2)
        d_diam = float(lbl.get("dot_diameter_mm", 1.5))
        d_sp = float(lbl.get("dot_spacing_mm", 2.5))
        c_sp = float(lbl.get("char_spacing_mm", 3.0))
        l_sp = float(lbl.get("line_spacing_mm", 4.0))
        # create sub-group for label
        sub_id = f"braille_{text.replace(' ', '_')}"
        sub = dwg.g(id=sub_id, **{"inkscape:groupmode":"layer", "inkscape:label":f"Braille: {text}"})
        # render braille group (relative coords)
        braille_group = render_braille_to_group(dwg, text,
                                                origin_mm=(0.0, 0.0),
                                                dot_diameter_mm=d_diam,
                                                dot_spacing_mm=d_sp,
                                                char_spacing_mm=c_sp,
                                                line_spacing_mm=l_sp)
        # translate group from centered coordinates to absolute svg coordinates
        # compute transform: origin centered (-w/2..w/2) -> svg coord
        ox_mm = pos[0] + fig_w_mm / 2.0   # center->svg x
        oy_mm = fig_h_mm / 2.0 - pos[1]   # center->svg y (invert)
        # append braille_group children into sub with translation
        # simplest: wrap braille_group into <g transform="translate(ox,oy)">
        trans = f"translate({ox_mm},{oy_mm})"
        sub.add(dwg.g(braille_group.elements, transform=trans))
        layer_braille.add(sub)
    dwg.add(layer_braille)

    # Save file
    dwg.save()
    print(f"SVG saved to: {output_svg}")

# -----------------------
# ENTRY POINT
# -----------------------
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python generate_svg_from_params.py params.json")
        sys.exit(1)
    params_path = sys.argv[1]
    params = load_params(params_path)
    build_svg_from_params(params)


FileNotFoundError: params file not found: --f=/run/user/1000/jupyter/runtime/kernel-v3ca1c05cfb4d3a4b723f5761633972bc3b4eaf142.json

# sjsjs

In [ ]:
"""
Generador de SVG con capas (grid, axes, curves, markers, braille).
- Usa svgwrite para crear un SVG físico en mm.
- Las capas son grupos <g> con atributos compatibles con Inkscape.
- Personaliza parámetros en la sección CONFIGURACIÓN.

Ejecutar:
    python generar_svg_capas.py

Salida:
    A5_output_layers.svg  (archivo SVG con capas)
"""

import math
import numpy as np
import svgwrite

# ---------------------------
# CONFIGURACIÓN (edita aquí)
# ---------------------------

# Tamaño de la figura en mm (ancho, alto)
fig_w_mm = 173.0
fig_h_mm = 113.0

# Conversión mm -> pulgada calibrada (tu preferencia: 23.4555555)
mm_per_inch = 23.4555555

# Rango de datos (ejes)
xlim = (-7.0, 7.0)
ylim = (-7.0, 7.0)

# Paso entre ticks en unidades de dato (ej: 0.5)
tick_step = 0.5

# Estética: grosores en mm
grid_stroke_mm = 0.25
axis_stroke_mm = 0.6
curve_stroke_mm = 0.9
marker_edge_stroke_mm = 0.2

# Marcadores: forma por función, tamaño en mm (diámetro para 'o', lado para 's', lado para triángulo)
marker_shapes = ['o', 's', '^']              # f1 -> circle, f2 -> square, f3 -> triangle
marker_sizes_mm = [3.0, 3.0, 3.5]           # tamaños de marcadores por función

# Curvas a dibujar: en formato de funciones lambda sobre numpy
functions = [
    lambda x: x,          # f(x) = x
    lambda x: x**2,       # f(x) = x^2
    lambda x: x**3        # f(x) = x^3
]
curve_styles = ["solid", "dash", "dot"]     # estilos (solo indicativos; en SVG uso stroke-dasharray)

# Muestreo continuo para curvas (cantidad de puntos)
n_curve_samples = 800

# Muestreo de marcadores (adaptativo simple: listas por función)
marker_xs = [
    np.linspace(xlim[0], xlim[1], 35),   # para f(x)=x
    np.concatenate([                       # para f(x)=x^2 (densidad variable)
        np.linspace(-7.0, -3.0, 1),
        np.linspace(-3.0, -2.0, 10),
        np.linspace(-2.0, -1.0, 7),
        np.linspace(-1.0,  1.0, 7),
        np.linspace(1.0, 2.0, 7),
        np.linspace(2.0, 3.0, 10),
        np.linspace(3.0, 7.0, 1)
    ]),
    np.concatenate([                       # para f(x)=x^3
        np.linspace(-7.0, -2.0, 1),
        np.linspace(-2.0, -1.5, 10),
        np.linspace(-1.5, -1.0, 6),
        np.linspace(-1.0,  1.0, 6),
        np.linspace(1.0, 1.5, 6),
        np.linspace(1.5, 2.0, 10),
        np.linspace(2.0, 7.0, 1)
    ])
]

# Braille: etiquetas a renderizar (lista de dicts)
# position_mm está en coordenadas de la placa: (x_mm, y_mm) con origen en el centro de la placa
braille_labels = [
    {
        "text": "Figura 1",
        "position_mm": (-70.0, -50.0),  # colocar en esquina inferior izquierda (centro=0,0)
        "dot_diameter_mm": 1.5,
        "dot_height_mm": 0.8,
        "dot_spacing_mm": 2.5,
        "char_spacing_mm": 3.0,
        "line_spacing_mm": 4.0
    }
]

# Nombre de archivo de salida
output_svg = "A5_output_layers.svg"

# ---------------------------
# FUNCIONES UTILITARIAS
# ---------------------------

def mm2in(mm, mm_per_inch=mm_per_inch):
    """Convertir mm a pulgadas usando factor calibrado (mm_per_inch)."""
    return mm / mm_per_inch

def data_to_svg_coords(x, y, xlim, ylim, width_mm, height_mm):
    """
    Mapea coordenadas de datos (x,y) al sistema SVG en mm.
    - El sistema SVG usado aquí tiene el origen en la esquina superior izquierda,
      x hacia la derecha y y hacia abajo.
    - Mapeamos el rectángulo [xlim,ylim] al rectángulo [0,width_mm] x [0,height_mm],
      centrando los datos en la placa.
    """
    x0, x1 = xlim
    y0, y1 = ylim
    # fracciones en [0,1]
    fx = (x - x0) / (x1 - x0)
    fy = (y - y0) / (y1 - y0)
    # SVG x: 0..width, SVG y: top..bottom so invertimos fy
    sx = fx * width_mm
    sy = (1 - fy) * height_mm
    return sx, sy

def svg_stroke_dash(style_name):
    """Devuelve stroke-dasharray para estilos simples."""
    if style_name == "solid":
        return None
    if style_name == "dash":
        return "6,3"
    if style_name == "dot":
        return "1,3"
    return None

# ---------------------------
# BRAILLE: mapping y render
# ---------------------------

# Mapa Braille para letras a-z (grade-1 básico)
BRAILLE_ALPHA = {
    'a': (1,), 'b': (1,2), 'c': (1,4), 'd': (1,4,5), 'e': (1,5),
    'f': (1,2,4), 'g': (1,2,4,5), 'h': (1,2,5), 'i': (2,4), 'j': (2,4,5),
    'k': (1,3), 'l': (1,2,3), 'm': (1,3,4), 'n': (1,3,4,5), 'o': (1,3,5),
    'p': (1,2,3,4), 'q': (1,2,3,4,5), 'r': (1,2,3,5), 's': (2,3,4), 't': (2,3,4,5),
    'u': (1,3,6), 'v': (1,2,3,6), 'w': (2,4,5,6), 'x': (1,3,4,6), 'y': (1,3,4,5,6), 'z': (1,3,5,6),
}
CAPITAL_SIGN = (6,)           # signo de mayúscula
NUMBER_SIGN  = (3,4,5,6)      # signo numérico
SPACE = ()
DIGIT_TO_ALPHA = { '1':'a','2':'b','3':'c','4':'d','5':'e','6':'f','7':'g','8':'h','9':'i','0':'j' }

def char_to_cells(ch):
    """Convertir un carácter a una lista de celdas Braille (cada celda = tupla de puntos)."""
    if ch == ' ':
        return [SPACE]
    if ch.isdigit():
        # se manejará por text_to_cells para secuencias numéricas
        return [SPACE]
    if ch.lower() in BRAILLE_ALPHA:
        if ch.isupper():
            return [CAPITAL_SIGN, BRAILLE_ALPHA[ch.lower()]]
        else:
            return [BRAILLE_ALPHA[ch.lower()]]
    # Por defecto: espacio (extender mapeo si necesitas más símbolos)
    return [SPACE]

def text_to_cells(text):
    """
    Convierte texto a una secuencia de celdas Braille:
    - Inserta NUMBER_SIGN al inicio de secuencia numérica
    - Inserta CAPITAL_SIGN antes de letra mayúscula
    """
    cells = []
    i = 0
    while i < len(text):
        ch = text[i]
        if ch.isdigit():
            # añadir signo numérico y consumir la secuencia de dígitos
            cells.append(NUMBER_SIGN)
            while i < len(text) and text[i].isdigit():
                cells.extend(char_to_cells(DIGIT_TO_ALPHA[text[i]]))
                i += 1
            continue
        cells.extend(char_to_cells(ch))
        i += 1
    return cells

def render_braille_to_svg(group, text, origin_mm, dot_diameter_mm=1.5, dot_spacing_mm=2.5,
                          char_spacing_mm=3.0, line_spacing_mm=4.0, fill_color="#000000"):
    """
    Dibuja el texto en Braille como círculos en el grupo SVG dado.
    - group: svgwrite container (g) donde se van a añadir los círculos.
    - origin_mm: (x_mm, y_mm) en coordenadas de la placa (centro->convertir antes).
    - Los cilindros 3D NO se representan en SVG; aquí solo se colocan los círculos (vista superior).
    """
    # convertir origin (centro coords) a coordenadas SVG:
    # data_to_svg_coords espera datos en escala de ejes (no usar aquí). Haremos mapeo directo:
    # asumimos origin_mm está en sistema centrado: (-w/2 .. w/2) -> (0..w)
    ox_mm, oy_mm = origin_mm
    # transformar a sistema SVG: top-left origin
    origin_svg_x = (ox_mm + fig_w_mm/2.0)
    origin_svg_y = (fig_h_mm/2.0 - oy_mm)

    # obtener celdas
    lines = text.split('\n')
    y = origin_svg_y
    for line in lines:
        cells = text_to_cells(line)
        cursor_x = origin_svg_x
        # dibujar cada celda
        for cell in cells:
            if cell == SPACE:
                cursor_x += char_spacing_mm
                continue
            # para cada punto en la celda
            for d in cell:
                # mapear índice d (1..6) a (col,row)
                # col: 0 -> dots 1,2,3 ; col 1 -> dots 4,5,6
                col = 0 if d in (1,2,3) else 1
                row_map = {1:0,2:1,3:2,4:0,5:1,6:2}
                row = row_map[d]
                # offsets relativos en mm
                x_offset = (col - 0.5) * dot_spacing_mm
                y_offset = (1 - row) * dot_spacing_mm  # row 0 top, row 2 bottom
                cx = cursor_x + x_offset
                cy = y + y_offset
                # crear círculo (vista superior)
                group.add(group.circle(center=(f"{cx}mm", f"{cy}mm"),
                                       r=f"{(dot_diameter_mm/2.0):.3f}mm",
                                       fill=fill_color,
                                       stroke="none"))
            cursor_x += char_spacing_mm
        # bajar línea
        y += line_spacing_mm

# ---------------------------
# CREAR SVG y CAPAS
# ---------------------------

# Inicializar documento SVG con tamaño físico en mm
dwg = svgwrite.Drawing(filename=output_svg, size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"), profile='tiny')

# Namespace para Inkscape layers (añadimos atributos en cada grupo)
inkscape_ns = "http://www.inkscape.org/namespaces/inkscape"

# — capa: fondo (placa) — (representa la placa/area)
layer_plate = dwg.g(id="plate", **{"inkscape:groupmode":"layer", "inkscape:label":"Plate"})
# rectángulo que representa la placa entera (color muy claro)
layer_plate.add(dwg.rect(insert=(0,0), size=(f"{fig_w_mm}mm", f"{fig_h_mm}mm"),
                         fill="#ffffff"))
dwg.add(layer_plate)

# — capa: grid —
layer_grid = dwg.g(id="grid", **{"inkscape:groupmode":"layer", "inkscape:label":"Grid"})
# calcular ticks en datos
xticks = np.arange(xlim[0], xlim[1] + 1e-9, tick_step)
yticks = np.arange(ylim[0], ylim[1] + 1e-9, tick_step)

# dibujar líneas verticales (x = const)
for xv in xticks:
    # convertir punto (xv, any y)
    sx1, sy1 = data_to_svg_coords(xv, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(xv, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
    layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#cccccc", stroke_width=f"{grid_stroke_mm}mm"))
# dibujar líneas horizontales (y = const)
for yv in yticks:
    sx1, sy1 = data_to_svg_coords(xlim[0], yv, xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(xlim[1], yv, xlim, ylim, fig_w_mm, fig_h_mm)
    layer_grid.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                            stroke="#eeeeee", stroke_width=f"{grid_stroke_mm}mm"))
dwg.add(layer_grid)

# — capa: axes (ejes principales) —
layer_axes = dwg.g(id="axes", **{"inkscape:groupmode":"layer", "inkscape:label":"Axes"})
# eje y = 0 (horizontal line across)
sx1, sy1 = data_to_svg_coords(xlim[0], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
sx2, sy2 = data_to_svg_coords(xlim[1], 0.0, xlim, ylim, fig_w_mm, fig_h_mm)
layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                        stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
# eje x = 0 (vertical line)
sx1, sy1 = data_to_svg_coords(0.0, ylim[0], xlim, ylim, fig_w_mm, fig_h_mm)
sx2, sy2 = data_to_svg_coords(0.0, ylim[1], xlim, ylim, fig_w_mm, fig_h_mm)
layer_axes.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                        stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
dwg.add(layer_axes)

# — capa: curves (curvas continuas) —
layer_curves = dwg.g(id="curves", **{"inkscape:groupmode":"layer", "inkscape:label":"Curves"})
# generar x continuo y dibujar cada curva como polyline
x_cont = np.linspace(xlim[0], xlim[1], n_curve_samples)
for i, f in enumerate(functions):
    ys = f(x_cont)
    # construir lista de puntos SVG en mm
    points = [ data_to_svg_coords(xv, yv, xlim, ylim, fig_w_mm, fig_h_mm) for xv, yv in zip(x_cont, ys) ]
    # convertir a lista de strings "xmm,ymm"
    points_str = [ (f"{px:.6f}mm", f"{py:.6f}mm") for px, py in points ]
    # crear polyline
    dash = svg_stroke_dash(curve_styles[i]) if i < len(curve_styles) else None
    stroke_kwargs = {"stroke": "#222222", "fill": "none", "stroke_width": f"{curve_stroke_mm}mm"}
    if dash:
        stroke_kwargs["stroke_dasharray"] = dash
    layer_curves.add(dwg.polyline(points=[(px,py) for px,py in points_str], **stroke_kwargs))
dwg.add(layer_curves)

# — capa: markers (cada marcador en su propia forma) —
layer_markers = dwg.g(id="markers", **{"inkscape:groupmode":"layer", "inkscape:label":"Markers"})
for i, f in enumerate(functions):
    xs = marker_xs[i]
    ys = f(xs)
    shape = marker_shapes[i] if i < len(marker_shapes) else 'o'
    size_mm = marker_sizes_mm[i] if i < len(marker_sizes_mm) else 3.0
    for xm, ym in zip(xs, ys):
        sx, sy = data_to_svg_coords(xm, ym, xlim, ylim, fig_w_mm, fig_h_mm)
        # dibujar formas SVG: circle, rect, triangle
        if shape == 'o':
            layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                         r=f"{(size_mm/2.0):.3f}mm",
                                         fill="#ffffff", stroke="#000000",
                                         stroke_width=f"{marker_edge_stroke_mm}mm"))
        elif shape == 's':
            # rect centered at sx,sy with side=size_mm
            half = size_mm/2.0
            x0 = sx - half
            y0 = sy - half
            # svgwrite espera coordenadas numéricas; usamos strings mm
            layer_markers.add(dwg.rect(insert=(f"{x0}mm", f"{y0}mm"),
                                       size=(f"{size_mm}mm", f"{size_mm}mm"),
                                       fill="#ffffff", stroke="#000000",
                                       stroke_width=f"{marker_edge_stroke_mm}mm"))
        elif shape == '^':
            # triángulo equilátero con lado=size_mm, centrado en (sx,sy)
            a = size_mm
            h = (math.sqrt(3)/2.0) * a
            # vértices relativos
            v1 = (sx, sy - 2*h/3.0)
            v2 = (sx - a/2.0, sy + h/3.0)
            v3 = (sx + a/2.0, sy + h/3.0)
            layer_markers.add(dwg.polygon(points=[(f"{v1[0]}mm", f"{v1[1]}mm"),
                                                  (f"{v2[0]}mm", f"{v2[1]}mm"),
                                                  (f"{v3[0]}mm", f"{v3[1]}mm")],
                                          fill="#ffffff", stroke="#000000", stroke_width=f"{marker_edge_stroke_mm}mm"))
        else:
            # por defecto, círculo
            layer_markers.add(dwg.circle(center=(f"{sx}mm", f"{sy}mm"),
                                         r=f"{(size_mm/2.0):.3f}mm",
                                         fill="#ffffff", stroke="#000000",
                                         stroke_width=f"{marker_edge_stroke_mm}mm"))
dwg.add(layer_markers)

# — capa: ticks (marcas en los ejes) —
layer_ticks = dwg.g(id="ticks", **{"inkscape:groupmode":"layer", "inkscape:label":"Ticks"})
tick_len_mm = 0.8
# marcas horizontales (y ticks) drawn as short lines across at x=0
for yv in yticks:
    sx1, sy1 = data_to_svg_coords(0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(-0.12, yv, xlim, ylim, fig_w_mm, fig_h_mm)
    layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                             stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
# marcas verticales (x ticks)
for xv in xticks:
    sx1, sy1 = data_to_svg_coords(xv, 0.12, xlim, ylim, fig_w_mm, fig_h_mm)
    sx2, sy2 = data_to_svg_coords(xv, -0.12, xlim, ylim, fig_w_mm, fig_h_mm)
    layer_ticks.add(dwg.line(start=(f"{sx1}mm", f"{sy1}mm"), end=(f"{sx2}mm", f"{sy2}mm"),
                             stroke="#000000", stroke_width=f"{axis_stroke_mm}mm"))
dwg.add(layer_ticks)

# — capa: Braille (cada etiqueta en su propio subgrupo) —
layer_braille = dwg.g(id="braille", **{"inkscape:groupmode":"layer", "inkscape:label":"Braille"})
for lbl in braille_labels:
    text = lbl.get("text", "")
    pos = lbl.get("position_mm", [0.0, 0.0])
    d_diam = lbl.get("dot_diameter_mm", 1.5)
    d_h = lbl.get("dot_height_mm", 0.8)
    d_sp = lbl.get("dot_spacing_mm", 2.5)
    c_sp = lbl.get("char_spacing_mm", 3.0)
    l_sp = lbl.get("line_spacing_mm", 4.0)
    # crear subgrupo por etiqueta (para que pueda ocultarse individualmente)
    sub = dwg.g(id=f"braille_{text.replace(' ','_')}", **{"inkscape:groupmode":"layer", "inkscape:label":f"Braille: {text}"})
    # renderizar círculos representando puntos en relieve
    render_braille_to_svg(sub, text, origin_mm=(pos[0], pos[1]), dot_diameter_mm=d_diam,
                          dot_spacing_mm=d_sp, char_spacing_mm=c_sp, line_spacing_mm=l_sp)
    layer_braille.add(sub)
dwg.add(layer_braille)

# ---------------------------
# GUARDAR SVG
# ---------------------------
# Escribir archivo (svgwrite guarda el archivo)
dwg.save()
print(f"SVG guardado en: {output_svg}")